In [6]:
import datetime
import quandl
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotnine as p9

In [8]:
quandl.ApiConfig.api_key = 'L5MEAxJ5KGy2Po9UWVXz'  

def grab_quandl_table(table_path, avoid_download=False, replace_existing=False, date_override=None, allow_old_file=False, **kwargs):
    """
    Downloads a Quandl table and returns the path to the downloaded file.

    Args:
        table_path (str): The path of the Quandl table to download.
        avoid_download (bool, optional): If True, avoids downloading the table if it already exists. Defaults to False.
        replace_existing (bool, optional): If True, replaces the existing file even if it has a non-zero size. Defaults to False.
        date_override (str, optional): The date to use for the downloaded file. If None, the current date is used. Defaults to None.
        allow_old_file (bool, optional): If True, allows using an old file if date_override is specified. Defaults to False.
        **kwargs: Additional keyword arguments to pass to the quandl.export_table function.

    Returns:
        str: The path to the downloaded file or "NoFileAvailable" if no file is available.

    Raises:
        None

    """
    root_data_dir = os.path.join(os.environ["HOME"], "quandl_data_table_downloads")
    data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
    
    if avoid_download and os.path.exists(data_symlink):
        print(f"Skipping any possible download of {table_path}")
        return data_symlink

    table_dir = os.path.dirname(data_symlink)
    if not os.path.isdir(table_dir):
        print(f'Creating new data dir {table_dir}')
        os.mkdir(table_dir)

    my_date = datetime.datetime.now().strftime("%Y%m%d") if date_override is None else date_override
    data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

    if os.path.exists(data_file):
        file_size = os.stat(data_file).st_size
        if replace_existing or not file_size > 0:
            print(f"Removing old file {data_file} size {file_size}")
        else:
            print(f"Data file {data_file} size {file_size} exists already, no need to download")
            return data_file

    dl = quandl.export_table(table_path, filename=data_file, api_key='L5MEAxJ5KGy2Po9UWVXz', **kwargs)
    file_size = os.stat(data_file).st_size
    if os.path.exists(data_file) and file_size > 0:
        print(f"Download finished: {file_size} bytes")
        if not date_override:
            if os.path.exists(data_symlink):
                print(f"Removing old symlink")
                os.unlink(data_symlink)
            print(f"Creating symlink: {data_file} -> {data_symlink}")
            os.symlink(data_file, data_symlink)
    else:
        print(f"Data file {data_file} failed download")
        return
    return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"

In [9]:
def fetch_quandl_table(table_path, avoid_download=True, **kwargs):
    return pd.read_csv(grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs))


def convert_expiration_code_to_date(code):
    """
    Converts the futures expiration code to a datetime.date object.

    Args:
        code (str): The expiration code representing the month and year.

    Returns:
        datetime.date: The corresponding date.

    Example:
        >>> convert_expiration_code_to_date('F21')
        datetime.date(2021, 1, 1)
    """
    month_codes = {
        'F': 1, 'G': 2, 'H': 3, 'J': 4, 'K': 5, 'M': 6,
        'N': 7, 'Q': 8, 'U': 9, 'V': 10, 'X': 11, 'Z': 12
    }
    return datetime.date(year=int('20' + code[1:]), month=month_codes[code[0]], day=1)

def fetch_contract_prices(exchange_code, future_code, option_code, start_date, end_date):
    contract_code = f"{exchange_code}_{future_code}_{option_code}"
    prices = []
    try:
        for year in range(start_date.year, end_date.year + 1):
            for quarter in ['H', 'M', 'U', 'Z']:
                data = quandl.get(f"OWF/{contract_code}_{year}{quarter}", start_date=start_date, end_date=end_date)
                prices.append(data)
    except quandl.errors.quandl_error.NotFoundError as e:
        print(f"Data not found for {contract_code}: {e}")
    except Exception as e:
        print(f"An error occurred while fetching contract prices: {e}")
    
    if prices:
        return pd.concat(prices)
    else:
        return pd.DataFrame()

In [10]:
def filter_shortest_expiry_contracts(data, start_date, end_date, exchange_code, future_code, option_code, days_expiration=30):
    """
    Filters contracts with the shortest expiry within the given date range that have at least days_expiration to expiry.
    Filters for specific contracts with given exchange_code and future_code.
    
    Args:
    - data: DataFrame containing contract data.
    - start_date (str): The start date of the range to filter contracts.
    - end_date (str): The end date of the range to filter contracts.
    - exchange_code (str): The exchange code to filter.
    - future_code (str): The future code to filter.
    - min_days_to_expiry (int): Minimum days to expiry for the contracts to be considered.
    
    Returns:
    - DataFrame: Filtered contracts with the shortest expiry for the given exchange_code and future_code.
    """
    # Convert string dates to datetime objects
    data['date'] = pd.to_datetime(data['date'])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter contracts within the date range and with at least min_days_to_expiry days to expiry
    filtered_data = data[
        (data['date'] >= start_date) & 
        (data['date'] <= end_date) & 
        (data['days_expiration'] >= days_expiration) &
        (data['exchange_code'] == exchange_code) &
        (data['futures_code'] == future_code) &
        (data['option_code'] == option_code)
    ]
    
    # Find the contract with the shortest expiry
    if not filtered_data.empty:
        shortest_expiry_contract = filtered_data.loc[filtered_data['days_expiration'].idxmin()]
        return shortest_expiry_contract
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no contracts match the criteria

In [11]:
def calculate_spread(fv_contract, tu_contract):
    if not fv_contract.empty and not tu_contract.empty:
        # Assuming 'futures' column has the price data
        if 'futures' in fv_contract and 'futures' in tu_contract:
            spread = fv_contract['futures'].iloc[0] - tu_contract['futures'].iloc[0] * 1.1066
            return spread
        else:
            raise ValueError("Missing 'futures' column in contract data.")
    else:
        raise ValueError("One or both contract data are empty.")


In [12]:
start_date = '2020-12-03'
end_date = '2023-08-31'
fv_contract = filter_shortest_expiry_contracts(data, start_date, end_date, 'CBT', 'FV')
tu_contract = filter_shortest_expiry_contracts(data, start_date, end_date, 'CBT', 'TU')

# Calculate spread if both contracts are available
if not fv_contract.empty and not tu_contract.empty:
    spread = calculate_spread(fv_contract, tu_contract)
    print(f"The spread between CBT-FV and CBT-TU*1.1066 is: {spread}")
else:
    print("Could not calculate spread because one or both contracts are missing.")

NameError: name 'data' is not defined

In [ ]:
t_ivm = fetch_quandl_table('AR/IVM', avoid_download=False)
t_ivm.head()
t_ivm.columns
t_ivs = fetch_quandl_table('AR/IVS', avoid_download=False)
print(t_ivs.shape)
t_ivs.iloc[[0,1,2,-3,-2,-1],:8]


Creating new data dir /Users/lazy/quandl_data_table_downloads/AR
Download finished: 290238259 bytes
Creating symlink: /Users/lazy/quandl_data_table_downloads/AR/IVM_20240115.zip -> /Users/lazy/quandl_data_table_downloads/AR/IVM_latest.zip


,exchange_code,futures_code,option_code,expiration,date,futures,atm,rr25,rr10,fly25,...,beta1,beta2,beta3,beta4,beta5,beta6,min_money,max_money,days_expiration,days_termination
0,CBT,BO,BO,V2024,2023-07-27,58.040000,0.303074,-0.004934,NaN,-0.005185,...,-0.013514,-0.210955,0.274140,1.697856,-1.023394,-4.179139,-0.549587,0.444217,421.24,445.0
1,ICE,KC,KC,H2026,2023-10-02,158.550000,0.288991,0.015911,NaN,0.000022,...,0.025836,-0.177754,0.101765,1.435220,-0.445698,-2.374568,-0.684043,0.532352,863.15,899.0
2,NYM,NG,NG,F2029,2023-07-27,4.703000,0.334631,NaN,NaN,NaN,...,0.159299,0.133878,-1.147995,-1.006677,1.590494,1.477700,-0.737270,0.754384,1979.00,1980.0
3,NYX,C,C,U2025,2023-10-02,2666.000000,0.231555,NaN,NaN,NaN,...,0.120610,-2.579713,1.159924,594.907525,-91.601577,-52350.067091,-0.074339,0.075473,696.80,714.0
4,CBT,BO,BO,1M,2023-07-27,67.346897,0.352890,-0.002929,-0.005976,0.002417,...,-0.024721,0.420288,-0.644045,23.556375,14.352899,-214.004951,-0.222364,0.238453,31.00,31.0
